#**FISHEASE**



##Import Libraries

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

##Download Dataset

In [3]:
!gdown 16KFubhf8kGBpfp3Fy1vQLJ7r8wyD1R7G

Downloading...
From: https://drive.google.com/uc?id=16KFubhf8kGBpfp3Fy1vQLJ7r8wyD1R7G
To: /content/Freshwater Fish Disease Aquaculture in south asia.zip
100% 26.1M/26.1M [00:00<00:00, 71.7MB/s]


In [4]:
!unzip "/content/Freshwater Fish Disease Aquaculture in south asia.zip"

Archive:  /content/Freshwater Fish Disease Aquaculture in south asia.zip
   creating: Freshwater Fish Disease Aquaculture in south asia/
  inflating: Freshwater Fish Disease Aquaculture in south asia/README.txt  
   creating: Freshwater Fish Disease Aquaculture in south asia/Test/
   creating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).jpeg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).png  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (10).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (11).jpg  
  inflating: F

##Constant

In [5]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
SEED = 42
CHANNELS = 3
EPOCHS = 50
PATH = "/content/Freshwater Fish Disease Aquaculture in south asia/"

##Training Image Preprocessing

In [6]:
training_set = tf.keras.utils.image_dataset_from_directory(
    PATH + 'Train/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 1747 files belonging to 7 classes.


##Validation Image Preprocessing

In [7]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    PATH + 'Test/',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)


Found 697 files belonging to 7 classes.


##Build Model

In [8]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input

In [9]:
num_classes = len(training_set.class_names)
print(f"Number of Classes: {num_classes}")


Number of Classes: 7


In [10]:

# Model Baru
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS) # Menggunakan IMAGE_SIZE dan CHANNELS yang sudah ada
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
base_model.trainable = False

custom_layers = Sequential([
    Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax') # Menggunakan num_classes
])

inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)
x = custom_layers(x)
model = Model(inputs, x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(training_set,
                    validation_data=validation_set,
                    epochs=EPOCHS)

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 712s 13s/step - accuracy: 0.3013 - loss: 5.4589 - val_accuracy: 0.7145 - val_loss: 0.8306
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.6423 - loss: 0.9760

KeyboardInterrupt: 

##Load Model

In [ ]:
model = load_model('model.keras')

In [ ]:
model.save('fishease_model.h5')

##Eval

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

test_dataset = image_dataset_from_directory(
    PATH + 'Test/',
    seed = 42,
    shuffle = True,
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
class_names = training_set.class_names
print(class_names)  # Untuk melihat daftar nama kelas

In [ ]:
all_predictions = []
all_labels = []
for images, labels in test_dataset:
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    all_predictions.extend(predicted_classes)
    all_labels.extend(labels.numpy())

##Confution Metrices

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_curve
)

accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')

print("Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)

In [ ]:
import seaborn as sns

conf_matrix = confusion_matrix(all_labels, all_predictions)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.savefig('Confusion Matrix.png', dpi = 300)
plt.show()

##Plotting the Accuracy and Loss Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('curves.png', dpi = 300)
plt.show()

##Inference

In [ ]:
def predict(model, image):
    img_array = tf.keras.preprocessing.image.img_to_array(image.numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.clf()
plt.figure(figsize=(8, 8))
for images, labels in test_dataset.take(6):
    for i in range(4):
        ax = plt.subplot(2, 2, i + 1)
        if i < len(images):
            ax.imshow(images[i].numpy().astype("uint8"))

            predicted_class, confidence = predict(model, images[i])
            actual_class = class_names[labels[i]]

            ax.set_title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
            ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
def sample_predict(model, image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    confidence = round(100 * (np.max(predictions[0])), 2)  # Get confidence

    plt.imshow(img)
    plt.title(f"Predicted class: {predicted_class}\nConfidence: {confidence}%")
    plt.axis("off")
    plt.show()

In [ ]:
sample_predict(model, '/content/ikan4.jpeg')

In [ ]:
!pip install requests pillow

In [ ]:

import requests
from PIL import Image
import io

def load_image_from_url(url):
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)
    image = Image.open(io.BytesIO(response.content))
    return image

In [ ]:
import tensorflow as tf

def preprocess_image(image):
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))  # Resize to match your model's input size
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = tf.expand_dims(image_array, 0)  # Create a batch
    return image_array

In [ ]:
def predict_from_url(model, url):
    image = load_image_from_url(url)
    image_array = preprocess_image(image)
    predictions = model.predict(image_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_names[predicted_class_index]
    confidence = round(100 * (np.max(predictions[0])), 2)  # Get confidence

    plt.imshow(image)
    plt.title(f"Predicted class: {predicted_class}\nConfidence: {confidence}%")
    plt.axis("off")
    plt.show()

    return predicted_class, confidence

In [ ]:
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRZYi-HghqZMgOuoqIYYFJKtcj2eYsts2EzWQ&s"  # Ganti dengan URL gambar Anda
predicted_class, confidence = predict_from_url(model, image_url)
print("Predicted class:", predicted_class)
print("Confidence:", confidence, "%")